In [1]:
import cv2
import numpy as np
from pytube import YouTube
import os
import urllib.request
from pytube.exceptions import VideoUnavailable
from urllib.error import HTTPError

In [2]:
# Output path directory
output_path = './trafficsystem'


In [3]:
# Ensure the directory exists
os.makedirs(output_path, exist_ok=True)

In [4]:

# Local path where the file will be saved
haar_cascade_path = os.path.join(output_path, "haarcascade_frontalface_default.xml")

In [5]:

# URL of the haarcascade_frontalface_default.xml file
url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"


In [6]:
# Download the file
urllib.request.urlretrieve(url, haar_cascade_path)
print(f"Downloaded haarcascade_frontalface_default.xml to {haar_cascade_path}")

Downloaded haarcascade_frontalface_default.xml to ./trafficsystem/haarcascade_frontalface_default.xml


In [7]:
# Load YOLO
def load_yolo(weights_path, config_path, classes_path):
    net = cv2.dnn.readNet(weights_path, config_path)
    with open(classes_path, "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    try:
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]
    except AttributeError:  # for older versions of OpenCV that do not have flatten()
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    return net, classes, output_layers, colors

def detect_objects(img, net, output_layers):
    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    return boxes, confidences, class_ids, indexes


In [8]:
def draw_labels(boxes, confidences, class_ids, classes, colors, img, indexes):
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [9]:


# Detect faces and save as PNG
def detect_and_save_faces(frame, face_cascade, output_folder, frame_count):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    for i, (x, y, w, h) in enumerate(faces):
        face = frame[y:y+h, x:x+w]
        face_filename = os.path.join(output_folder, f"frame{frame_count}_face{i}.png")
        cv2.imwrite(face_filename, face)

In [10]:

def process_video(video_path, weights_path, config_path, classes_path, face_cascade_path, output_folder):
    net, classes, output_layers, colors = load_yolo(weights_path, config_path, classes_path)
    cap = cv2.VideoCapture(video_path)
    face_cascade = cv2.CascadeClassifier(face_cascade_path)
    
    if not cap.isOpened():
        print("Error: Failed to open video file.")
        return

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Detect objects in the frame (YOLO)
        boxes, confidences, class_ids, indexes = detect_objects(frame, net, output_layers)
        draw_labels(boxes, confidences, class_ids, classes, colors, frame, indexes)

        # Detect and save faces
        detect_and_save_faces(frame, face_cascade, output_folder, frame_count)

        cv2.imshow('Video Processing', frame)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [11]:

def download_video(url, output_path):
    try:
        yt = YouTube(url)
        stream = yt.streams.filter(file_extension='mp4').get_highest_resolution()
        stream.download(output_path=output_path)
        print(f"Downloaded: {yt.title}")
        return os.path.join(output_path, stream.default_filename)
    except VideoUnavailable:
        print(f"Video {url} is unavailable.")
    except HTTPError as e:
        if e.code == 410:
            print(f"HTTP Error 410: The requested resource {url} is gone.")
        else:
            print(f"HTTP Error {e.code}: {e.reason}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
face_output_folder

In [12]:
def main(video_url, weights_path, config_path, classes_path, face_cascade_path):
    output_path = './downloaded_videos'
    os.makedirs(output_path, exist_ok=True)
   # video_path = download_video(video_url, output_path)
    video_path = './video.mp4/1.mp4'
    if video_path:
        face_output_folder = os.path.join(output_path, "faces")
        os.makedirs(face_output_folder, exist_ok=True)
        process_video(video_path, weights_path, config_path, classes_path, face_cascade_path, face_output_folder)


In [ ]:
if __name__ == "__main__":
    video_url = 'https://www.youtube.com/watch?v=XufeJTJbAGY'  # Replace with your YouTube video URL
    yolo_weights = 'yolov3.weights'  # Replace with your YOLO weights file
    yolo_config = 'yolov3.cfg'  # Replace with your YOLO config file
    yolo_classes = 'coco.names'  # Replace with your classes file
    face_cascade = os.path.join(output_path, 'haarcascade_frontalface_default.xml')  # Path to Haar cascade for face detection
    main(video_url, yolo_weights, yolo_config, yolo_classes, face_cascade)




2024-06-22 09:44:49.907 Python[29534:834821] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.
